In [16]:
from gensim.models import KeyedVectors
from tqdm import tqdm
import numpy as np
import sys

In [44]:
glove = '/home/manni/embs/gloveW2V.6B.300d.txt'
w2v = '/home/manni/embs/word2vec-google-news-300.gz'
wiki2v = '/home/manni/embs/enwiki_20180420_300d.txt.bz2'
dconf = ''

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(glove, binary=False)

In [45]:
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=True)

In [ ]:
wiki_model = KeyedVectors.load_word2vec_format(wiki2v, binary=False)

In [ ]:
words = wiki_model.most_similar('bank',topn=100)

In [ ]:
wiki_model.most_similar('bank',topn=100)

In [ ]:
#w2v_model.vocab

In [ ]:
def total_overlap(word,model,topn=100):
    overlap_matrix = np.zeros((topn,topn),dtype=np.int8)
    words = model.most_similar(word,topn=topn)
    words = set([i[0] for i in words])
    nns = dict()
    for word in words:
        _words = model.most_similar(word,topn=5)
        _words = set([i[0] for i in _words])
        nns[word]=_words
    for i,(k,a) in enumerate(nns.items()):
        for j,(kk,b) in enumerate(nns.items()):
            if k == kk:
                overlap_matrix[i][j]=1
                continue
            overlap = len(a & b)
            overlap_matrix[i][j]=overlap
    return nns,overlap_matrix

In [ ]:
nns,ovm = total_overlap('bank',glove_model,topn=100)

In [ ]:
def print_closer(n):
    print(voc[n])
    print(':::::')
    nn = set()
    for i,val in enumerate(ovm[n]>0):
        if val:
            print(i,voc[i])
            nn.add(voc[i])
    return nn

In [ ]:
a = print_closer(3)
b = print_closer(15)
print(a&b)

In [ ]:
clusters = dict()
voc = list(nns.keys())
for v in voc:
    clusters[v]=-1
c = 0
for i in range(len(ovm)):
    for j in range(len(ovm)):
        if i == j:
            continue
        vec = ovm[i]
        vec = vec>0
        vec_ = ovm[j]
        vec_= vec_>0
        list_a = [voc[i_] for i_,x in enumerate(vec) if x]
        list_b = [voc[i_] for i_,x in enumerate(vec_) if x]
        overlap = len(set(list_a) & set(list_b))
        if overlap>3:
            #print(i,':',voc[i],j,':',voc[j])
            #if i == 10 or j == 10:
            #    print(i,j,clusters[voc[i]],clusters[voc[j]])
            if clusters[voc[i]]>-1 and clusters[voc[j]]>-1:
                c_ = min(clusters[voc[i]],clusters[voc[j]])
                clusters[voc[i]]=c_
                clusters[voc[j]]=c_
            elif clusters[voc[i]]>-1:
                clusters[voc[j]]=clusters[voc[i]]
            elif clusters[voc[j]]>-1:
                clusters[voc[i]]=clusters[voc[j]]
            else:
                clusters[voc[i]]=c
                clusters[voc[j]]=c
                c+=1
            #if i == 10 or j == 10:
            #    print(i,j,clusters[voc[i]],clusters[voc[j]])
        #if np.array_equal(vec,vec_):
        #    print(i,':',voc[i],j,':',voc[j])
    #print('***')

In [ ]:
clusters

In [ ]:
clusters_ = dict()
for i,c in enumerate(clusters.values()):
    if c == -1:
        continue
    if c in clusters_:
        clusters_[c].append(voc[i])
    else:
        clusters_[c] = [voc[i]]

In [ ]:
clusters_

In [ ]:
def get_matrix(model,vocab):
    matrix = list()
    for word in vocab:
        vec = model.get_vector(word)
        matrix.append(vec)
    return matrix

In [ ]:
matrix = get_matrix(glove_model,vocab)

In [ ]:
matrix = np.asarray(matrix)

# validate embeddings

# clustering

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

In [ ]:
db = DBSCAN(eps=0.4,min_samples=2,metric='cosine').fit(matrix)

In [ ]:
labels = db.labels_
clusters = dict()
for i,c in enumerate(labels):
    if c == -1:
        continue
    if c in clusters:
        clusters[c].append(vocab[i])
    else:
        clusters[c] = [vocab[i]]

In [ ]:
clusters

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [ ]:
X = [[1, 2], [2, 2], [2, 3],[8, 7], [8, 8], [25, 80]]
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering.labels_

# wordnet

In [5]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
import re
from collections import Counter

In [6]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [ ]:
wn.synsets('dog')[0].hyponyms()

In [ ]:
wn.synset('dog.n.01').definition()

In [ ]:
wn.synset('frump.n.01').definition()

In [ ]:
wn.synset('dog.n.03').definition()

In [ ]:
dog = wn.synset('dog.n.01')
dog.hypernyms()

In [ ]:
dog.hyponyms()

In [ ]:
wn.synsets('bank')[9].hyponyms()

In [7]:
vocab = list(wn.all_synsets())

In [8]:
len(vocab)

117659

In [9]:
vocab_ = set()
for ss in vocab:
    word = ss.name().split('.')[0]
    if '_' in word:
        continue
    vocab_.add(word)

In [10]:
len(vocab_)

58736

In [11]:
list(vocab_)[:10]

['',
 'abelmosk',
 'waterdog',
 'aspidophoroides',
 'massenet',
 'dolefully',
 'unoiled',
 'infomercial',
 'paradoxical',
 'inability']

In [12]:
def get_uniques(input_list):
    uniques = set()
    for k,v in Counter(input_list).items():
        if v>1:
            continue
        uniques.add(k)
    return uniques

In [13]:
import spacy

nlp = spacy.load("en_core_web_sm")

2022-09-30 12:21:27.003676: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-09-30 12:21:28.716575: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-09-30 12:21:28.717119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-09-30 12:21:28.717464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0b:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-09-30 12:21:28.717798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 2 with properties: 
pciBusID: 0000:16:0

In [35]:
worddesc_raw = dict()
for _word in tqdm(vocab_,position=0):
    word = lemmatizer.lemmatize(_word)
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        if word in worddesc_raw:
            worddesc_raw[word].append(txt)
        else:
            worddesc_raw[word]=[txt]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58736/58736 [00:01<00:00, 41739.81it/s]


In [36]:
sv.save(worddesc_raw,'worddesc_raw')

Saved the data


In [14]:
nouns = False

In [17]:
lemmatizer = WordNetLemmatizer()
#ps = PorterStemmer()
stops = set(stopwords.words('english'))
word_desc = dict()
for _word in tqdm(vocab_,position=0):
    word = lemmatizer.lemmatize(_word)
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        if nouns:
            doc = nlp(txt)
            try:
                txt = [token.text for token in doc if token.pos_=='NOUN']
                txt = ' '.join(txt)
            except:
                print(txt)
                sys.exit()
        txt = re.sub(r'\W+', ' ', txt)
        words = [w for w in txt.split()]
        #txts = synset.examples()
        #txt = ' '.join(txts)
        #txt = re.sub(r'\W+', ' ', txt)
        #words_ = [w for w in txt.split()]
        #words = words + words_
        hypo = synset.hyponyms()
        hypo_words = [ss.name().split('.')[0] for ss in hypo]
        hyper = synset.hypernyms()
        hyper_words = [ss.name().split('.')[0] for ss in hyper]
        _words = words+hypo_words+hyper_words
        words = list()
        for w in _words:
            words.extend(w.split('_'))
        words = [lemmatizer.lemmatize(w) for w in words if w not in stops]
        words = set(words)
        #add to dict
        if word in word_desc:
            word_desc[word].append(words)
        else:
            word_desc[word]=[words]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58736/58736 [00:11<00:00, 4963.10it/s]


In [18]:
len(word_desc['dog'])

8

In [40]:
word2desc = dict()
for k,v in word_desc.items():
    word2desc[k] = list()
    all_words = [item for sublist in v for item in sublist]
    #uniques = get_uniques(all_words)
    for words in v:
        words = set(words)
        #word2desc[k].append(words&uniques)
        word2desc[k].append(words)

In [23]:
word2desc['growth']

[{'amelogenesis',
  'angiogenesis',
  'apposition',
  'auxesis',
  'biological',
  'biology',
  'blossoming',
  'cenogenesis',
  'changing',
  'cohesion',
  'cultivation',
  'culture',
  'cycle',
  'cytogenesis',
  'event',
  'foliation',
  'fructification',
  'gametogenesis',
  'germination',
  'gradually',
  'habit',
  'individual',
  'infructescence',
  'intussusception',
  'involved',
  'juvenescence',
  'level',
  'life',
  'masculinization',
  'morphogenesis',
  'myelinization',
  'neurogenesis',
  'organic',
  'organically',
  'organism',
  'palingenesis',
  'psychogenesis',
  'psychomotor',
  'psychosexual',
  'purely',
  'rooting',
  'simple',
  'suppression',
  'teething',
  'teratogenesis',
  'unfolding'},
 {'form', 'progression', 'simpler'},
 {'accession',
  'accretion',
  'becoming',
  'growth',
  'important',
  'larger',
  'longer',
  'multiplication',
  'numerous',
  'population',
  'pullulation',
  'relaxation',
  'widening'},
 set(),
 {'beginning', 'coming', 'forth', '

In [21]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [41]:
sv.save(word2desc,"word2desc")

Saved the data


In [ ]:
word2desc['anarchism'][0]

In [ ]:
print(word2desc['fast'])
print(len(word2desc['fast']))

In [ ]:
def verbose(word):
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        txt = re.sub(r'\W+', ' ', txt)
        print(txt)
        hyper = synset.hypernyms()
        hyper_words = [ss.name().split('.')[0] for ss in hyper]
        print(hyper_words)
        hypo = synset.hyponyms()
        hypo_words = [ss.name().split('.')[0] for ss in hypo]
        print(hypo_words)
        print('______')

In [ ]:
lemmatizer.lemmatize("faster")

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp("a part of a river where the current is very fast")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
verbose('dogging')

In [ ]:
verbose('rapid')

In [ ]:
verbose('fast')

In [ ]:
word2desc['rapid']

In [ ]:
word2desc['fast']

In [ ]:
def get_overlap(a,b):
    a = word2desc[a]
    b = word2desc[b]
    overlap = 0
    match = None
    for i,x in enumerate(a):
        for j,y in enumerate(b):
            if len(x & y) > overlap:
                overlap = len(x & y)
                match = (i,j)
    return match, overlap

In [ ]:
get_overlap('fast','rapid')

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
print(ps.stem('moving'))
print(lemmatizer.lemmatize('moving'))

In [ ]:
word2desc['authority'][4]

In [ ]:
len(vocab_)

In [ ]:
list(vocab_)[:10]

In [ ]:
dog.examples()

In [ ]:
for i in vocab_[100:110]:
    print(i)
    txt = i.definition().lower()
    print(txt)
    txt = re.sub(r'\W+', ' ', txt)
    print(txt)
    words = [word for word in txt.split() if word not in stops]
    print(set(words))
    word = i.name().split('.')[0]
    print(wn.synsets(word))
    print()

# odd one out

In [54]:
def odd_word_out(input_words,model):
    '''The function accepts a list of word and returns the odd word.'''
     
    # Generate all word embeddings for the given list of words
    vocab = model.vocab
     
    whole_word_vectors = [model.get_vector(i) for i in input_words if i in vocab]
     
    # average vector for all word vectors
    mean_vector = np.mean(whole_word_vectors,axis=0)
     
    # Iterate over every word and find similarity
    odd_word = None
    minimum_similarity = 99999.0 # Can be any very high value
     
    for i in input_words:
        if i in model.vocab:
            similarity = model.cosine_similarities(model.get_vector(i),[mean_vector])
            if similarity < minimum_similarity:
                minimum_similarity = similarity
                odd_word = i

            print("cosine similarity score between %s and mean_vector is %.3f"%(i,similarity))
     
    print("\nThe odd word is: "+odd_word)

In [55]:
input_words = {'current', 'fast', 'part', 'river', 'waterway'}

In [56]:
odd_word_out(input_words,w2v_model)

cosine similarity score between river and mean_vector is 0.797
cosine similarity score between waterway and mean_vector is 0.795
cosine similarity score between current and mean_vector is 0.298
cosine similarity score between fast and mean_vector is 0.428
cosine similarity score between part and mean_vector is 0.375

The odd word is: current


In [59]:
w2v_model.similar_by_word('current',30)

[('Current', 0.5574568510055542),
 ('currrent', 0.512985348701477),
 ('thecurrent', 0.49619051814079285),
 ('curent', 0.4640691876411438),
 ('future', 0.462577223777771),
 ('existing', 0.44827771186828613),
 ('new', 0.4458693861961365),
 ('previous', 0.4399641454219818),
 ('normalized_FFO_guidance', 0.41918817162513733),
 ('prevailing', 0.4187409281730652),
 ('currently', 0.40886324644088745),
 ('speaker_Rauhi_Fattouh', 0.40469616651535034),
 ('NeoPharm_anticipates', 0.40467962622642517),
 ('changesin', 0.40295684337615967),
 ('instrument_SHORT_TERM', 0.39964842796325684),
 ('the', 0.3985964059829712),
 ('examines_Chinas', 0.3956565856933594),
 ('Visit_www.otc_advisors.com', 0.3910021185874939),
 ('coincident_index_measuring', 0.3904479146003723),
 ('assumes_noobligation', 0.38776859641075134),
 ('continuation', 0.3865417242050171),
 ('Dr_Dulare', 0.3858053386211395),
 ('xi_unanticipated', 0.3855990767478943),
 ('projected', 0.38018572330474854),
 ('regarding_Bachoco', 0.37681156396865